In [2]:
import pandas as pd
import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report

from RandomForestClassifierGridSearch import RandomForestClassifierGridSearch
import pickle

c:\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
scores = pd.read_parquet('../data/processed/scores_test.parquet')
approvals = pd.read_parquet('../data/processed/approvals.parquet')
scores.shape

(7263, 23)

In [4]:
cotas_columns = [col for col in scores.columns if 'classificacao' in col]
cotas_columns.pop(0) # removing 'classificacao_final_universal'
cotas_columns

['classificacao_final_cotas_negros',
 'classificacao_final_publicas1',
 'classificacao_final_publicas2',
 'classificacao_final_publicas3',
 'classificacao_final_publicas4',
 'classificacao_final_publicas5',
 'classificacao_final_publicas6',
 'classificacao_final_publicas7',
 'classificacao_final_publicas8']

In [5]:
scores['cotista'] = scores[cotas_columns].notnull().any(axis=1).astype(int)

In [6]:
for column in cotas_columns:
    colum_name = re.sub("classificacao_final_", "", f'{column}_flag')
    scores[colum_name] = scores[column].notnull().astype(int)

In [7]:
flags_columns = list(scores.columns[scores.columns.str.contains('flag')])

In [8]:
flags_columns

['cotas_negros_flag',
 'publicas1_flag',
 'publicas2_flag',
 'publicas3_flag',
 'publicas4_flag',
 'publicas5_flag',
 'publicas6_flag',
 'publicas7_flag',
 'publicas8_flag']

In [9]:
df = pd.merge(scores, approvals, how='left', on='numero_inscricao', indicator=True)

In [10]:
df[df.curso.notna()][['numero_inscricao', 'nome_x','nome_y', 'course', 'curso']]

,numero_inscricao,nome_x,nome_y,course,curso
4,20188170,Ana Beatriz Cattermol Cavalcante,Ana Beatriz Cattermol Cavalcante,DIURNO ADMINISTRAÇÃO (BACHARELADO),Administração (Bacharelado)
6,20183676,Ana Clara Marques da Silva,Ana Clara Marques da Silva,DIURNO ADMINISTRAÇÃO (BACHARELADO),Administração (Bacharelado)
9,20170011,Ana Paula Nunes Bezerra,Ana Paula Nunes Bezerra,DIURNO ADMINISTRAÇÃO (BACHARELADO),Administração (Bacharelado)
24,20184514,Bruno dos Santos Fernandes,Bruno dos Santos Fernandes,DIURNO ADMINISTRAÇÃO (BACHARELADO),Administração (Bacharelado)
29,20181503,Carina da Silva Ferreira,Carina da Silva Ferreira,DIURNO ADMINISTRAÇÃO (BACHARELADO),Administração (Bacharelado)
...,...,...,...,...,...
7257,20176727,Maria Clara de Melo Pierre,Maria Clara de Melo Pierre,CAMPUS UNB PLANALTINA (FUP) – NOTURNO CIÊNCIAS...,Ciências Naturais (Licenciatura)
7258,21270449,Sarah de Oliveira Nascimento,Sarah de Oliveira Nascimento,CAMPUS UNB PLANALTINA (FUP) – NOTURNO CIÊNCIAS...,Ciências Naturais (Licenciatura)
7259,20102521,Eloisa Graziele Rodrigues de Ara ujo,Eloisa Graziele Rodrigues de Araujo,GESTÃO AMBIENTAL (BACHARELADO),Gestão Ambiental (Bacharelado)
7260,20190553,Julia de Sousa Vale,Julia de Sousa Vale,GESTÃO AMBIENTAL (BACHARELADO),Gestão Ambiental (Bacharelado)


In [11]:
df['label'] = df._merge.apply(lambda x: 1 if x == 'both' else 0)

In [12]:
FEATURES = ['escore_bruto_p1_etapa1',
            'escore_bruto_p2_etapa1',
            'nota_redacao_etapa1',
            'escore_bruto_p1_etapa2',
            'escore_bruto_p2_etapa2',
            'nota_redacao_etapa2',
            'escore_bruto_p1_etapa3',
            'escore_bruto_p2_etapa3',
            'nota_redacao_etapa3',
            'argumento_final']

In [13]:
def convert_string_to_float(df, colnames):
    for colname in colnames:
        df[colname] = df[colname].str.replace(' ', "", regex=True)
        df[colname] = df[colname].str.replace('[R$]', "", regex=True)
        df[colname] = df[colname].str.replace(',', ".", regex=False)
        df[colname] = df[colname].apply(float)
    return df

In [14]:
df = convert_string_to_float(df, FEATURES)

In [15]:
course_dummies = pd.get_dummies(df.course)
course_dummies_columns = list(course_dummies.columns)
df = pd.concat([df, course_dummies], axis=1)

In [16]:
FEATURES.extend(course_dummies_columns)

In [17]:
len(FEATURES)

101

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7263 entries, 0 to 7262
Columns: 130 entries, numero_inscricao to TURISMO (BACHARELADO)
dtypes: category(1), float64(12), int32(10), int64(1), object(15), uint8(91)
memory usage: 2.5+ MB


## Baseline Model

In [19]:
X = df[FEATURES] # features
y = df['label'] # labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=47) # 70% training and 30% test

In [20]:
# Create a Random Forest Classifier
model = RandomForestClassifier(random_state=47)

# Fit randomized search
model = model.fit(X_train, y_train)

y_pred = model.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print('Accuracy:', metrics.accuracy_score(y_test, y_pred))
classification_report(y_test, y_pred, output_dict=True)

Accuracy: 0.848589125946318


{'0': {'precision': 0.8591954022988506,
  'recall': 0.980327868852459,
  'f1-score': 0.9157733537519143,
  'support': 1220},
 '1': {'precision': 0.6065573770491803,
  'recall': 0.15879828326180256,
  'f1-score': 0.25170068027210885,
  'support': 233},
 'accuracy': 0.848589125946318,
 'macro avg': {'precision': 0.7328763896740155,
  'recall': 0.5695630760571309,
  'f1-score': 0.5837370170120115,
  'support': 1453},
 'weighted avg': {'precision': 0.8186829040998326,
  'recall': 0.848589125946318,
  'f1-score': 0.8092840675022277,
  'support': 1453}}

## Baseline Model + cotista

In [21]:
FEATURES.append('cotista')

In [22]:
len(FEATURES)

102

In [23]:
X = df[FEATURES] # features
y = df['label'] # labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=47) # 70% training and 30% test

In [24]:
# Create a Random Forest Classifier
model = RandomForestClassifier(random_state=47)

# Fit randomized search
model = model.fit(X_train, y_train)

y_pred = model.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print('Accuracy:', metrics.accuracy_score(y_test, y_pred))
classification_report(y_test, y_pred, output_dict=True)

Accuracy: 0.8623537508602891


{'0': {'precision': 0.8701015965166908,
  'recall': 0.9827868852459016,
  'f1-score': 0.9230177059276367,
  'support': 1220},
 '1': {'precision': 0.72,
  'recall': 0.2317596566523605,
  'f1-score': 0.35064935064935066,
  'support': 233},
 'accuracy': 0.8623537508602891,
 'macro avg': {'precision': 0.7950507982583455,
  'recall': 0.6072732709491311,
  'f1-score': 0.6368335282884936,
  'support': 1453},
 'weighted avg': {'precision': 0.8460316226774692,
  'recall': 0.8623537508602891,
  'f1-score': 0.8312339297543121,
  'support': 1453}}

## Baseline Model + Flags

In [25]:
FEATURES.extend(flags_columns)
len(FEATURES)

111

In [29]:
X = df[FEATURES] # features
y = df['label'] # labelss
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=47) # 70% training and 30% test

In [27]:
# Create a Random Forest Classifier
model = RandomForestClassifier(random_state=47)

# Fit randomized search
model = model.fit(X_train, y_train)

y_pred = model.predict(X_test)
classification_report(y_test, y_pred, output_dict=True)

{'0': {'precision': 0.8802045288531775,
  'recall': 0.9877049180327869,
  'f1-score': 0.9308613364233295,
  'support': 1220},
 '1': {'precision': 0.8214285714285714,
  'recall': 0.296137339055794,
  'f1-score': 0.4353312302839117,
  'support': 233},
 'accuracy': 0.8768066070199587,
 'macro avg': {'precision': 0.8508165501408744,
  'recall': 0.6419211285442905,
  'f1-score': 0.6830962833536206,
  'support': 1453},
 'weighted avg': {'precision': 0.8707793409110348,
  'recall': 0.8768066070199587,
  'f1-score': 0.8513991790038633,
  'support': 1453}}

## Hyperparameter tuning + class_weight

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300, 500],      # Number of trees in the forest
    'max_depth': [5, 10, 20, 40, 60],            # Maximum depth of each tree
    'min_samples_split': [5, 10, 20, 40, 60],       # Minimum number of samples required to split an internal node
    'min_samples_leaf': [2, 8, 16, 32]          # Minimum number of samples required to be at a leaf node
}

In [ ]:
# Assuming you have your training data X_train and corresponding labels y_train

# Instantiate the class
rf_gs = RandomForestClassifierGridSearch(X_train, y_train)

# Fit the classifier using GridSearchCV
rf_gs.fit(param_grid)

# Assuming you have your test data X_test and corresponding labels y_test

# Evaluate the best model on the test data
report = rf_gs.evaluate(X_test, y_test)
print("Classification Report:")
print(report)

NameError: name 'RandomForestClassifierGridSearch' is not defined

In [ ]:
with open('model_flags_tuned.pickle','wb') as f:
    pickle.dump(model, f)

In [ ]:
#with open('model_flags_tuned.pickle','wb') as f:
#  pickle.dump(model, f)

## Stratified KFold + Hyperameter tuning + Flags

In [26]:
FEATURES.remove('argumento_final')
print(len(FEATURES))

X = df[FEATURES] # features
y = df['label'] # labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=47) # 70% training and 30% test

110


In [36]:
# # Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],      # Number of trees in the forest
    'max_depth': [5, 10, 20, 40, 80],            # Maximum depth of each tree
    'min_samples_split': [5, 10, 20, 50],       # Minimum number of samples required to split an internal node
    'min_samples_leaf': [2, 8, 16, 32],          # Minimum number of samples required to be at a leaf node
}

In [37]:
X_train.shape

(5810, 110)

In [38]:
# Assuming you have your training data X_train and corresponding labels y_train

# Instantiate the class
rf_gs = RandomForestClassifierGridSearch(X_train, y_train)

# Fit the classifier using GridSearchCV
rf_gs.fit(param_grid)

# Assuming you have your test data X_test and corresponding labels y_test

# Evaluate the best model on the test data
report = rf_gs.evaluate(X_test, y_test)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      1220
           1       0.57      0.64      0.60       233

    accuracy                           0.86      1453
   macro avg       0.75      0.77      0.76      1453
weighted avg       0.87      0.86      0.87      1453



In [39]:
rf_gs.save_model('stratified_kfold_classweight15_tuned_model_spelling')

In [27]:
with open('../ml_dev/models/stratified_kfold_classweight15_tuned_model_spelling.pickle', 'rb') as f:
  model_loaded = pickle.load(f)

In [28]:
y_pred = model_loaded.predict(X_test)
classification_report(y_test, y_pred, output_dict=True)

{'0': {'precision': 0.9286912751677853,
  'recall': 0.9073770491803279,
  'f1-score': 0.9179104477611939,
  'support': 1220},
 '1': {'precision': 0.5670498084291188,
  'recall': 0.6351931330472103,
  'f1-score': 0.5991902834008097,
  'support': 233},
 'accuracy': 0.8637302133516862,
 'macro avg': {'precision': 0.747870541798452,
  'recall': 0.7712850911137691,
  'f1-score': 0.7585503655810018,
  'support': 1453},
 'weighted avg': {'precision': 0.8706992161518806,
  'recall': 0.8637302133516862,
  'f1-score': 0.8668011578121441,
  'support': 1453}}

In [74]:
def predict_approval(model, new_data):
    
    # Predict diabetes
    predictions = model.predict_proba(new_data)
    approval_prob = round(predictions[0][1], ndigits=3)

    #pred_to_label = {0: 'Negative', 1: 'Positive'}

    # Make a list of predictions
    #data = []
    #for t, pred in zip(new_data, predictions):
    #    data.append({'prediction': pred[0]})

    return approval_prob

In [63]:
FEATURES

['escore_bruto_p1_etapa1',
 'escore_bruto_p2_etapa1',
 'nota_redacao_etapa1',
 'escore_bruto_p1_etapa2',
 'escore_bruto_p2_etapa2',
 'nota_redacao_etapa2',
 'escore_bruto_p1_etapa3',
 'escore_bruto_p2_etapa3',
 'nota_redacao_etapa3',
 'AGRONOMIA (BACHARELADO)',
 'ARQUITETURA E URBANISMO (BACHARELADO)',
 'ARQUIVOLOGIA (BACHARELADO)',
 'ARTES CÊNICAS - INTERPRETAÇÃO TEATRAL (BACHARELADO)',
 'ARTES VISUAIS (BACHARELADO)',
 'ARTES VISUAIS (LICENCIATURA)',
 'BIBLIOTECONOMIA (BACHARELADO)',
 'BIOTECNOLOGIA (BACHARELADO)',
 'CAMPUS UNB CEILÂNDIA (FCE) ENFERMAGEM (BACHARELADO)',
 'CAMPUS UNB PLANALTINA (FUP) – DIURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
 'CAMPUS UNB PLANALTINA (FUP) – NOTURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
 'CIÊNCIA DA COMPUTAÇÃO (BACHARELADO)',
 'CIÊNCIA POLÍTICA (BACHARELADO)',
 'CIÊNCIAS AMBIENTAIS (BACHARELADO)',
 'CIÊNCIAS BIOLÓGICAS (BACHARELADO)',
 'CIÊNCIAS CONTÁBEIS (BACHARELADO)',
 'CIÊNCIAS ECONÔMICAS (BACHARELADO)',
 'CIÊNCIAS SOCIAIS – ANTROPOLOGIA / SOCIOLOGIA 

In [42]:
covariables = list({"escore_bruto_p1_etapa1": 5.172,
                            "escore_bruto_p2_etapa1": 14.653,
                            "nota_redacao_etapa1": 6.947,
                            "escore_bruto_p1_etapa2": 3.845,
                            "escore_bruto_p2_etapa2": 19.994,
                            "nota_redacao_etapa2": 7.222,
                            "escore_bruto_p1_etapa3": 4.998,
                            "escore_bruto_p2_etapa3": 16.66,
                            "nota_redacao_etapa3": 8.06,
                            "argumento_final": -25.699,
                            "2.1.1 CAMPUS  DARCY RIBEIRO – DIURNO  ADMINISTRAÇÃO (BACHARELADO)": 1.0,               
                            "cotista": 0.0,
                            "cotas_negros_flag": 0.0,
                            "publicas1_flag": 0.0,
                            "publicas2_flag": 0.0,
                            "publicas3_flag": 0.0,
                            "publicas4_flag": 0.0,
                            "publicas5_flag": 0.0,
                            "publicas6_flag": 0.0,
                            "publicas7_flag": 0.0,
                            "publicas8_flag": 0.0}.keys())

In [30]:
from samples import sample_approved, sample_not_approved

In [43]:
{'teste':sample_not_approved.get('cotista')}

{'teste': 0.0}

In [36]:
list(sample_not_approved.keys())

['escore_bruto_p1_etapa1',
 'escore_bruto_p2_etapa1',
 'nota_redacao_etapa1',
 'escore_bruto_p1_etapa2',
 'escore_bruto_p2_etapa2',
 'nota_redacao_etapa2',
 'escore_bruto_p1_etapa3',
 'escore_bruto_p2_etapa3',
 'nota_redacao_etapa3',
 'argumento_final',
 '2.1.1 CAMPUS  DARCY RIBEIRO – DIURNO  ADMINISTRAÇÃO (BACHARELADO)',
 'cotista',
 'cotas_negros_flag',
 'publicas1_flag',
 'publicas2_flag',
 'publicas3_flag',
 'publicas4_flag',
 'publicas5_flag',
 'publicas6_flag',
 'publicas7_flag',
 'publicas8_flag']

In [76]:
new_sample = sample_not_approved
#new_sample = pd.DataFrame([{col: new_sample.get(col) for col in FEATURES}])
#new_sample = new_sample.fillna(0)

In [79]:
[True for key in new_sample.keys() if key in FEATURES else False]

SyntaxError: invalid syntax (Temp/ipykernel_17636/1785515755.py, line 1)

In [80]:
[True for key in new_sample.keys()]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [72]:
new_sample.to_dict()

{'escore_bruto_p1_etapa1': {0: 5.172},
 'escore_bruto_p2_etapa1': {0: 14.653},
 'nota_redacao_etapa1': {0: 6.947},
 'escore_bruto_p1_etapa2': {0: 3.845},
 'escore_bruto_p2_etapa2': {0: 19.994},
 'nota_redacao_etapa2': {0: 7.222},
 'escore_bruto_p1_etapa3': {0: 4.998},
 'escore_bruto_p2_etapa3': {0: 16.66},
 'nota_redacao_etapa3': {0: 8.06},
 'AGRONOMIA (BACHARELADO)': {0: 0},
 'ARQUITETURA E URBANISMO (BACHARELADO)': {0: 0},
 'ARQUIVOLOGIA (BACHARELADO)': {0: 0},
 'ARTES CÊNICAS - INTERPRETAÇÃO TEATRAL (BACHARELADO)': {0: 0},
 'ARTES VISUAIS (BACHARELADO)': {0: 0},
 'ARTES VISUAIS (LICENCIATURA)': {0: 0},
 'BIBLIOTECONOMIA (BACHARELADO)': {0: 0},
 'BIOTECNOLOGIA (BACHARELADO)': {0: 0},
 'CAMPUS UNB CEILÂNDIA (FCE) ENFERMAGEM (BACHARELADO)': {0: 0},
 'CAMPUS UNB PLANALTINA (FUP) – DIURNO CIÊNCIAS NATURAIS (LICENCIATURA)': {0: 0},
 'CAMPUS UNB PLANALTINA (FUP) – NOTURNO CIÊNCIAS NATURAIS (LICENCIATURA)': {0: 0},
 'CIÊNCIA DA COMPUTAÇÃO (BACHARELADO)': {0: 0},
 'CIÊNCIA POLÍTICA (BACHAREL

In [64]:
FEATURES[33]

'DIURNO ADMINISTRAÇÃO (BACHARELADO)'

In [65]:
sample_not_approved.get(FEATURES[33])

In [75]:
predict_approval(model_loaded, new_data=new_sample)

0.104

In [60]:
X_test.columns.to_list()

['escore_bruto_p1_etapa1',
 'escore_bruto_p2_etapa1',
 'nota_redacao_etapa1',
 'escore_bruto_p1_etapa2',
 'escore_bruto_p2_etapa2',
 'nota_redacao_etapa2',
 'escore_bruto_p1_etapa3',
 'escore_bruto_p2_etapa3',
 'nota_redacao_etapa3',
 'AGRONOMIA (BACHARELADO)',
 'ARQUITETURA E URBANISMO (BACHARELADO)',
 'ARQUIVOLOGIA (BACHARELADO)',
 'ARTES CÊNICAS - INTERPRETAÇÃO TEATRAL (BACHARELADO)',
 'ARTES VISUAIS (BACHARELADO)',
 'ARTES VISUAIS (LICENCIATURA)',
 'BIBLIOTECONOMIA (BACHARELADO)',
 'BIOTECNOLOGIA (BACHARELADO)',
 'CAMPUS UNB CEILÂNDIA (FCE) ENFERMAGEM (BACHARELADO)',
 'CAMPUS UNB PLANALTINA (FUP) – DIURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
 'CAMPUS UNB PLANALTINA (FUP) – NOTURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
 'CIÊNCIA DA COMPUTAÇÃO (BACHARELADO)',
 'CIÊNCIA POLÍTICA (BACHARELADO)',
 'CIÊNCIAS AMBIENTAIS (BACHARELADO)',
 'CIÊNCIAS BIOLÓGICAS (BACHARELADO)',
 'CIÊNCIAS CONTÁBEIS (BACHARELADO)',
 'CIÊNCIAS ECONÔMICAS (BACHARELADO)',
 'CIÊNCIAS SOCIAIS – ANTROPOLOGIA / SOCIOLOGIA 

In [69]:
my_sample = pd.DataFrame([X_test.iloc[0]])
#my_sample.rename_ax

In [71]:
my_sample.rename(columns = {'AGRONOMIA (BACHARELADO)': 'AGRONOMIA 123 (BACHARELADO)'})

,escore_bruto_p1_etapa1,escore_bruto_p2_etapa1,nota_redacao_etapa1,escore_bruto_p1_etapa2,escore_bruto_p2_etapa2,nota_redacao_etapa2,escore_bruto_p1_etapa3,escore_bruto_p2_etapa3,nota_redacao_etapa3,AGRONOMIA 123 (BACHARELADO),...,cotista,cotas_negros_flag,publicas1_flag,publicas2_flag,publicas3_flag,publicas4_flag,publicas5_flag,publicas6_flag,publicas7_flag,publicas8_flag
510,1.149,21.549,5.808,3.845,25.857,8.273,1.428,25.228,8.148,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
my_sample.columns.to_list()

['escore_bruto_p1_etapa1',
 'escore_bruto_p2_etapa1',
 'nota_redacao_etapa1',
 'escore_bruto_p1_etapa2',
 'escore_bruto_p2_etapa2',
 'nota_redacao_etapa2',
 'escore_bruto_p1_etapa3',
 'escore_bruto_p2_etapa3',
 'nota_redacao_etapa3',
 'AGRONOMIA (BACHARELADO)',
 'ARQUITETURA E URBANISMO (BACHARELADO)',
 'ARQUIVOLOGIA (BACHARELADO)',
 'ARTES CÊNICAS - INTERPRETAÇÃO TEATRAL (BACHARELADO)',
 'ARTES VISUAIS (BACHARELADO)',
 'ARTES VISUAIS (LICENCIATURA)',
 'BIBLIOTECONOMIA (BACHARELADO)',
 'BIOTECNOLOGIA (BACHARELADO)',
 'CAMPUS UNB CEILÂNDIA (FCE) ENFERMAGEM (BACHARELADO)',
 'CAMPUS UNB PLANALTINA (FUP) – DIURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
 'CAMPUS UNB PLANALTINA (FUP) – NOTURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
 'CIÊNCIA DA COMPUTAÇÃO (BACHARELADO)',
 'CIÊNCIA POLÍTICA (BACHARELADO)',
 'CIÊNCIAS AMBIENTAIS (BACHARELADO)',
 'CIÊNCIAS BIOLÓGICAS (BACHARELADO)',
 'CIÊNCIAS CONTÁBEIS (BACHARELADO)',
 'CIÊNCIAS ECONÔMICAS (BACHARELADO)',
 'CIÊNCIAS SOCIAIS – ANTROPOLOGIA / SOCIOLOGIA 

In [58]:
model_loaded.predict(new_sample)

array([0], dtype=int64)